# Purpose

Try out [bidict](https://bidict.readthedocs.io/en/main/home.html) - a bidirectional mapping library for Python. Provides several friendly, efficient data structures for working with bidirectional mappings in Python. It's like a 2-way dict where the keys and values can be used as keys.

In [1]:
from bidict import bidict

In [2]:
regular_dict = {
    0: (0, 0),
    1: (1, 0),
    2: (2, 0),
    3: (3, 0),
    4: (0, 1),
    5: (1, 1),
    6: (2, 1),
    7: (3, 1),
    8: (0, 2),
    9: (1, 2),
    10: (2, 2),
    11: (2, 3),
}
regular_dict

{0: (0, 0),
 1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (0, 1),
 5: (1, 1),
 6: (2, 1),
 7: (3, 1),
 8: (0, 2),
 9: (1, 2),
 10: (2, 2),
 11: (2, 3)}

In [3]:
test_bidict = bidict(regular_dict)
test_bidict

bidict({0: (0, 0), 1: (1, 0), 2: (2, 0), 3: (3, 0), 4: (0, 1), 5: (1, 1), 6: (2, 1), 7: (3, 1), 8: (0, 2), 9: (1, 2), 10: (2, 2), 11: (2, 3)})

In [4]:
print(regular_dict[1])
print(test_bidict[1])
print(test_bidict.inverse[(1, 0)])

(1, 0)
(1, 0)
1


# Compare speed of bidict with function call

In [5]:
import itertools
import random

## Linear index with a `lambda` expression

In [6]:
p = lambda m, n, Nx: n * Nx + m
p_inv = lambda pp, Nx: (pp // Nx, pp % Nx)

print(p(1, 1, 4))
print(p_inv(5, 4))

5
(1, 1)


## Linear index with a regular function

In [7]:
def p_func(m, n, Nx):
    return n * Nx + m


def p_inv_func(pp, Nx):
    return (pp // Nx, pp % Nx)


print(p_func(1, 1, 4))
print(p_inv_func(5, 4))

5
(1, 1)


## Generate random unique `m,n` coordinates and corresponding `p` coordinates

In [8]:
N = 100
nums = [i for i in range(N)]

# Generate random p values
random_nums = [i for i in range(N * N)]
random.shuffle(random_nums)

# Generate all possible (m,n) permuations
ordered_pairs = [(m, n) for n in nums for m in nums]

# Randomly shuffle these pairs in place
random_pairs = ordered_pairs.copy()
random.shuffle(random_pairs)

print(len(random_nums))
print(len(ordered_pairs))
print(len(random_pairs))
# print(ordered_pairs)
# print(random_pairs)

10000
10000
10000


## Bidict

In [9]:
temp = {i: v for i, v in enumerate(ordered_pairs)}
# print(temp)
coords = bidict(temp)
print(len(coords))
# print(coords)
coords.inverse[(0, 1)]
# coords.inverse[(47, 55)]

10000


100

## Timings

### `lambda` expression

#### (m,n) &rarr; p

In [10]:
%%timeit
for a in random_pairs:
    temp = p(*a, N)

1.9 ms ± 16.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### p &rarr; (m,n)

In [11]:
%%timeit
for a in random_nums:
    temp = p_inv(a, N)

1.17 ms ± 94.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Function

#### (m,n) &rarr; p

In [12]:
%%timeit
for a in random_pairs:
    temp = p_func(*a, N)

2.54 ms ± 479 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### p &rarr; (m,n)

In [13]:
%%timeit
for a in random_nums:
    temp = p_inv_func(a, N)

1.09 ms ± 8.96 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Bidict

#### (m,n) &rarr; p

In [14]:
%%timeit
for a in random_pairs:
    temp = coords.inverse[a]

2.89 ms ± 451 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### p &rarr; (m,n)

In [15]:
%%timeit
for a in random_nums:
    temp = coords[a]

1.73 ms ± 356 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Conclusions

It looks like there is little speed penalty in using a `bidict` instead of a `lambda` expression or function for corresponding a linear index with an `(m,n)` index. The advantage of the `bidict` is that it is just as easy to express a complex relationship between the linear index and an `(m,n)` index as a simple relationship, whereas it may be quite a bit more complicated to do so with a function.